<a href="https://colab.research.google.com/github/9-coding/DeepLearningZeroToAll/blob/main/Lab05_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Lab-05-3 Logistic Regression
# 영상 코드 말고 아래의 댓글을 참조함.

import tensorflow as tf
import numpy as np

x_train = np.array([
    [1, 2],
    [2, 3],
    [3, 1],
    [4, 3],
    [5, 3],
    [6, 2]], dtype=np.float32)
y_train = np.array([
    [0],
    [0],
    [0],
    [1],
    [1],
    [1]], dtype=np.float32)

x_test = np.array([[5, 2]], dtype=np.float32)
y_test = np.array([[1]], dtype=np.float32)

dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))
W = tf.Variable(tf.zeros([2, 1]), name='weight')
b = tf.Variable(tf.zeros([1]), name='bias')

# 원소의 자료구조 반환
dataset.element_spec

# 회귀식 설정
def logistic_regression(features):
    hypothesis = tf.sigmoid(tf.matmul(features, W) + b)
    return hypothesis

def loss_fn(features, labels):
    hypothesis = logistic_regression(features)
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

def grad(hypothesis, features, labels):
    with tf.GradientTape() as tape:
        loss_value = loss_fn(features, labels)
    return tape.gradient(loss_value, [W,b])

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

EPOCHS = 3000

for step in range(EPOCHS + 1):
    for features, labels in iter(dataset):
        hypothesis = logistic_regression(features)
        grads = grad(hypothesis, features, labels)
        optimizer.apply_gradients(grads_and_vars=zip(grads, [W,b]))
        if step % 300 == 0:
            print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(features, labels)))
            # print(hypothesis)

def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
    return accuracy

test_acc = accuracy_fn(logistic_regression(x_test), y_test)
print('Accuracy: {}%'.format(test_acc * 100))

Iter: 0, Loss: 0.6874
Iter: 300, Loss: 0.5054
Iter: 600, Loss: 0.4535
Iter: 900, Loss: 0.4228
Iter: 1200, Loss: 0.3992
Iter: 1500, Loss: 0.3790
Iter: 1800, Loss: 0.3608
Iter: 2100, Loss: 0.3442
Iter: 2400, Loss: 0.3288
Iter: 2700, Loss: 0.3146
Iter: 3000, Loss: 0.3013
Accuracy: 100%


### 데이터 세팅
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))<br>
- tf.data.Dataset 파이프라인을 이용하여 값을 입력
- from_tensor_slices 클래스 매서드를 사용하면 리스트, 넘파이, 텐서플로우 자료형에서 데이터셋을 만들 수 있음


### <br>def logistic_regression(features):
- 시그모이드 함수 설정

### <br>def loss_fn(features, labels):
- 비용 반환 함수.
- labels : 실제값 | hypothesis : 예측값.<br>
- cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + (1 - labels) * tf.math.log(1 - hypothesis))
  * 실제 레이블이 0과 1일 때를 설정하여 실제 레이블에 가까우면 줄어들도록 손실을 구함.
  * 두 항을 더하고 평균을 구해 전체 데이터의 손실 계산.

### <br> def grad(hypothesis, features, labels):
- 기울기를 구하며 변수의 정보들을 tape에 기록.

### <br> 학습
optimizer.apply_gradients(grads_and_vars=zip(grads, [W,b]))
-  epoch를 돌면서 optimizer를 통해 지속적으로 최적값 update. -> 최적값을 찾아감.

### <br>def accuracy_fn(hypothesis, labels):
- 0.5를 threshold로 잡고 예측값을 판단한 다음, 실제값과 일치하는지 확인.